# `f3dasm`: Framework for Data-Driven Design and Analysis of Structures and Materials 
*April 3rd, 2023* <br>
*Code Release Week \#1*

# Table of contents

**1. Project introduction**
- 1.1 Overview
- 1.2 Computational Framework
- 1.3 Installation
- 1.4 Getting started

**2. Demonstration**

## 1.1 Overview

f3dasm is an attempt to unite data-driven design and analysis of structures of materials.
More concretely, 

## 1.2 Computational framework

`f3dasm` is one python package that consists of 8 submodules:

**Use `f3dasm` to handle your design of experiments**

Modules:
- `f3dasm.design`
- `f3dasm.experiment`

**Use `f3dasm` to compare models**

Modules:
- `f3dasm.machinelearning`
- `f3dasm.optimization`
- `f3dasm.sampling`


**Use `f3dasm` to generate data**

Modules:
- `f3dasm.functions`
- `f3dasm.data`
- `f3dasm.simulation`


## 1.3 Installation

### System requirements
`f3dasm` is purely Python code and compatible with:
1. Python 3.7 to 3.10.
2. the three major operations system (Linux, MacOS, Ubuntu).
3. the default environment of Google Colab (Python 3.8, Linux) 
4. the `pip` package manager system.

Installation instruction can be found in the documentation page under [Getting Started](https://bessagroup.github.io/F3DASM/gettingstarted.html)

We install the full version of `f3dasm`:

In [6]:
# Weird hot-fix only for Google Colab
%pip install omegaconf==1.4.0

try:
    import f3dasm
except ModuleNotFoundError:
    %pip install -i https://test.pypi.org/simple/ f3dasm==0.9.2
    import f3dasm

  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.2.3
    Uninstalling omegaconf-2.2.3:
      Successfully uninstalled omegaconf-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hydra-core 1.2.0 requires omegaconf~=2.2, but you have omegaconf 1.4.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


You can view the version and it's dependencies:

In [7]:
f3dasm.show_versions()


f3dasm:
        0.9.2

System:
    python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:35)  [GCC 10.4.0]
executable: /home/martin/miniconda3/envs/f3dasm_env3/bin/python
   machine: Linux-5.14.0-1059-oem-x86_64-with-glibc2.10

Core package dependencies:
        numpy: 1.23.5
        scipy: 1.9.3
       pandas: 1.5.2
   matplotlib: 3.6.2
       pathos: 0.3.0
        hydra: 1.2.0
     autograd: unknown

Machine learning extension:
   tensorflow: 2.11.0

Optimization extension:
       GPyOpt: 1.2.6
          GPy: 1.10.0
   tensorflow: 2.11.0
        pygmo: 2.19.0

Sampling extension:
        SALib: 1.4.7


#### Distinction between python package and repository
- The Python  PyPI package (`pip install f3dasm`) contains the code that is used when installing the package as a **user**. It contains only the `main` branch version.
- The GitHub repository is mainly for **developers** and besides the package includes:
  - Studies (more on that later)
  - Test suite
  - Documentation source
  - Tutorial notebooks

##  1.4 Getting started

The package contains a lot of implementation for each of the blocks. However, the installation `f3dasm` is modular: you decide what you want to use or not.

We can distinguish 3 ways of using `f3dasm`:

#### 1. Using `f3dasm` to handle your design of experiments
*Have your own functions and modules and coat them in a `f3dasm` sauce to manage and scale-up your experiments!*

The **core** package: contains the minimal installation to use `f3dasm` without extended features. 
Installed with `pip install f3dasm`

The core package contains the following features:
1. provide a way to parametrize your experiment with the **design-of-experiments** classes.
2. provide the option to investigate their experiment by **sampling** and **optimizing** their design.
3. provide the user guidance in **parallelizing** their program and ordering their data.
4. give the user ways of deploying their **experiment** at the HPC (TORQUE system)

The core package requires the following dependencies:
- `numpy` and `scipy`: for numerical operations
- `pandas`: for the representation of the design of experiments
- `matplotlib`: for plotting
- `hydra-core`: for deploying your experiment 
- `pathos`: for multiprocessing
- `autograd`: for computing gradients

#### 2. Using `f3dasm` to benchmark or compare models
*Go fully `f3dasm`: use existing implementations to benchmark parts of the data-driven machine learning process!*

You can solely use the core package, but it is advised to enrich `f3dasm` with its **extensions** 

The extensions contain the following features:
1. provide various **implementations** to accommodate common machine learning workflows.
2. provide **adapter** classes that link common machine learning libraries to `f3dasm` base classes. 


For each of the blocks, extensions can be installed to extend the choice of implementations. Installed with `pip install f3dasm[<name of extension>]`

The following extensions are available:
- **machinelearning**: containing various `tensorflow` related models
- **sampling**: containing sampling strategies from `SALib`
- **optimization**: containing various optimizers from `GPyOpt`, `pygmo` and `tensorflow`


#### 3. Develop on `f3dasm`
*Work hard, play hard: work towards making your implementations an official `f3dasm` extension!*

If you want your implementation to be part of the `f3dasm` package, you can develop an adapter and/or implementation for `f3dasm`

The **developement** package: contains the full installation plus requirements for developing on `f3dasm`. 
Installed with `pip install f3dasm[dev]`

Information on how to contribute to `f3dasm` can be found [on the wiki page of the GitHub repository](https://github.com/bessagroup/F3DASM/wiki)!

## 2. Demonstration

In the past practical sessions, I have shown you how to use `f3dasm` to benchmark various parts of a data-driven machine learning process (**use-case #2**).

Today I will show you how to use `f3dasm` to streamline your own data-driven process (**use-case #1**)

Import some other packages and set a seed

In [ ]:
import numpy as np
import logging
from pathos.helpers import mp  # For multiprocessing!
import time # For ... timing!

SEED = 42
np.random.seed(SEED)

### Example: Set up your program with f3dasm

Let's say we have a program that we want to execute. It is important that this could be **anything**. Like:
- Calculate the loss of some compliance curve in topology optimization
- Computing the mean stress and strain from some abaqus simulation
- Benchmarking various regressors in a multi-fidelity setting

At the top level of your experiment, you will probably have a main function that accepts some arguments and returns the quantity of interest.

Let's create such a function, just for demonstration purposes.

In [ ]:
def main(a: float, b: float, c: float) -> float:    
    functions = [f3dasm.functions.Rastrigin, f3dasm.functions.Levy, f3dasm.functions.Ackley]
    y = []
    for func in functions:
        f = func(dimensionality=3, scale_bounds=np.tile([-1.,1.], (3,1)), seed=SEED)
        time.sleep(.1)
        y.append(f(np.array([a,b,c])).ravel()[0])

    # Sum the values
    out = sum(y)
    logging.info(f"Executed program with a={a:.3f}, b={b:.3f}, c={c:.3f}: \t Result {out:.3f}")
    return out

What are we seeing:
- The program requires three floating points and returns a float as well.
- It creates three 3D-benchmark functions, evaluates them sequentially and sums the results
- We simulate some computational cost (0.1 seconds per evaluation) by calling the `time.sleep()` method
- We write to a log

> Note: `my_own_program` uses the integrated benchmark functions from `f3dasm`, but this could very well be one of your codes without any dependency on `f3dasm`.

Executing multiple experiments is easy:

In [ ]:
inputs = np.random.uniform(size=(10,3))

start_time = time.time()
outputs = np.array([main(*input_vals) for input_vals in inputs])
time_not_parallel = time.time() - start_time

print(f"It took {time_not_parallel:.5f} seconds to execute this for loop")

We can save the values of `outputs` for later use

This process (`main.py`) can be described with the following figure:

<img src="img/sequential.png" alt="alt text" width="30%" height="30%">

### Local parallelization

If you are familiar with [multiprocessing](https://docs.python.org/3/library/multiprocessing.html), you might already know that we can speed-up this function by parellizing the internal for loop:

We create a multiprocessing pool (`mp.Pool()`) where we map the functions to cores in our machine:

In [ ]:
def main_parallel(a: float, b: float, c: float) -> float:
    def evaluate_function(func, a, b, c):
        f = func(dimensionality=3, scale_bounds=np.tile([-1.,1.], (3,1)))
        y = f(np.array([a,b,c])).ravel()[0]
        time.sleep(.1)
        return y

    functions = [f3dasm.functions.Rastrigin, f3dasm.functions.Levy, f3dasm.functions.Ackley]
    with mp.Pool() as pool:
        y = pool.starmap(evaluate_function, [(func, a, b, c) for func in functions])

    # Sum the values
    out = sum(y)

    logging.info(f"Executed program with a={a:.3f}, b={b:.3f}, c={c:.3f}: \t Result: {out:.3f}")
    return out

Executing this function will speed up the process

In [ ]:
inputs = np.random.uniform(size=(10,3))

start_time = time.time()
outputs = np.array([main_parallel(*input_vals) for input_vals in inputs])
time_parallel = time.time() - start_time

print(f"It took {time_parallel:.5f} seconds to execute this for loop")
print(f"We are {time_not_parallel-time_parallel:.5f} seconds faster by parellelization!")

This process (`main_parallel.py`) can be described with the following figure:

<img src="img/parallel.png" alt="alt text" width="30%" height="30%">

### Scale-up: challenges

Now we would like to really scale things up. 

Q) What challenges lie along the way?

I asked ChatGPT:

- **1. Experiment design and analysis**: As the complexity of the experiment increases, it becomes more difficult to design experiments that are robust and reproducible, and to analyze the results in a meaningful way. This can lead to issues with experimental design, parameter tuning, and statistical analysis.

- **2. Parallelization**: As experiments become larger, it may be necessary to parallelize or distribute the computations across multiple machines or nodes in order to reduce the overall runtime. This introduces additional challenges such as synchronization between distributed processes.

- **3. Managing data**: As the volume of data generated by an experiment increases, it becomes more difficult to manage and store that data. This can lead to issues with data corruption, loss, or inconsistency.

This is where `f3dasm` is a helping hand!

#### 1. Experiment design and analysis

We can create a `f3dasm.DesignSpace` to capture the variables of interest:
- A `f3dasm.DesignSpace` consists of an input and output list of `f3dasm.Parameter` objects

In [ ]:
param_a = f3dasm.ContinuousParameter(name='a', lower_bound=-1., upper_bound=1.)
param_b = f3dasm.ContinuousParameter(name='b', lower_bound=-1., upper_bound=1.)
param_c = f3dasm.ContinuousParameter(name='c', lower_bound=-1., upper_bound=1.)
param_out = f3dasm.ContinuousParameter(name='y')

design = f3dasm.DesignSpace(input_space=[param_a, param_b, param_c], output_space=[param_out])

We can create an object to store the experiments: `f3dasm.ExperimentData`, but we can also **sample from this designspace**
We do that with the `f3dasm.sampling` submodule:

> Note that this submodule offers an extension (`f3dasm[sampling]`) that include sampling strategies from `SALib` 

In [ ]:
# Create the sampler object
sampler = f3dasm.sampling.RandomUniform(design=design, seed=SEED)

data: f3dasm.ExperimentData = sampler.get_samples(numsamples=10)

The data object is under the hood a pandas dataframe:

In [ ]:
data.data

The `y` values are NaN because we haven't evaluate our experiment yet! Let's do that:

Handy: we can retrieve the input columns of a specific row as a dictionary

In [ ]:
data.get_inputdata_by_index(index=3)

Unpacking the values as arguments of our experiment creates the same results:

In [ ]:
for index in range(data.get_number_of_datapoints()):
    value = main_parallel(**data.get_inputdata_by_index(index))
    data.set_outputdata_by_index(index, value)

Now our data-object is filled

In [ ]:
data.data

This process can be described with the following figure:

<img src="img/single_node.png" alt="alt text" width="50%" height="50%">

`f3dasm` can handle the experiment distribution. 

In order to set this up, navigate to a folder where you want to create your experiment and run `f3dasm.experiment.quickstart()`:

In [ ]:
# I'll not run this command because this is a demo

# f3dasm.experiment.quickstart()

This creates the following files and folders:

```
└── my_experiment 
    ├── main.py
    ├── config.py
    ├── config.yaml
    ├── default.yaml
    ├── pbsjob.sh
    └── README.md
    └── hydra/job_logging
        └── custom_script.py
```

Without going to much in detail, the following things have already been set up automatically:

**Logging**
- `hydra` (and the `custom_script.py`) take care of all (multiprocess) logging
- including writing across nodes when executing arrayjobs!

**Parameter storage**
- `config.yaml`, `config.py` and `default.yaml` can be used for easy reproducibility and parameter tuning of your experiment!

**Parallelization**
- `pbsjob.sh` can be used to execute your `main.py` file on the HPC, including array-jobs.

example:
```
qsub pbsjob.sh
qsub pbsjob.sh -t 0-10
```

**Saving data**
- `hydra` creates a new `outputs/<HPC JOBID>/` directory that saves all output files, logs and settings when executing `main.py`
- When executing arrayjobs, all arrayjobs write to the same folder!

#### 2. Parallelization

Let's recall: our single node process with `f3dasm.ExperimentData` can be abstracted by the following image:

<img src="single_node.png" alt="alt text" width="50%" height="50%">

Parallelizing the **outer loop** is more difficult, but we can do that across nodes with help of the `f3dasm.experiment.JobQueue`


In [ ]:
job_queue = f3dasm.experiment.JobQueue(filename='my_jobs')


We can fill the queue with the rows of the `f3dasm.ExperimentData` object:

In [ ]:
job_queue.create_jobs_from_experimentdata(data)
job_queue

10 jobs have been added and they are all up for grabs!

Let's first write this to disk so multiple nodes can access it:

In [ ]:
job_queue.write_new_jobfile()

A node can grab the first available job in the queue with the `get()` method:
The file is locked when accessing the information from the JSON file


In [ ]:
job_id = job_queue.get()
print(f"The first open job_id is {job_id}!")

After returning the `job_id`, the lock is removed and the job is changed to `in progress`

In [ ]:
job_queue.get_jobs()

When a new node asks a new job, it will return the next open job in line!

In [ ]:
job_id = job_queue.get()
print(f"The first open job_id is {job_id}!")

When a job is finished, you can mark it finished or with an error:

In [ ]:
job_queue.mark_finished(index=0)
job_queue.mark_error(index=1)

job_queue.get_jobs()

We can now change our simple script to handle multiprocessing across nodes!

In [ ]:
job_queue = f3dasm.experiment.JobQueue(filename='my_jobs2')
job_queue.create_jobs_from_experimentdata(data)

job_queue.write_new_jobfile()

data.store('data')

while True:
    try:
        jobnumber = job_queue.get()
    except f3dasm.experiment.NoOpenJobsError:
        break
    
    data = f3dasm.design.load_experimentdata('data')
    args = data.get_inputdata_by_index(jobnumber)

    value = main_parallel(**data.get_inputdata_by_index(jobnumber))
    data.set_outputdata_by_index(jobnumber, value)

    data.store('data')

    job_queue.mark_finished(jobnumber)

data.data

This process looks like this:

<img src="img/jobqueue.png" alt="alt text" width="50%" height="50%">

### 3. Managing data

Sometimes you don't want to write directly to the `ExperimentData` file. Perhaps the output is not a simple set of values, or you want to do some post-processing.
This is where the `f3dasm.Filehandler` comes in handy.

<img src="img/jobqueue_filehandler.png" alt="alt text" width="80%" height="80%">